In [38]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode
import spacy
import nltk
import stanza
from transformers import AutoTokenizer
from flair.data import Sentence
from flair.models import SequenceTagger
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
from huggingface_hub import login

In [39]:
nlp = spacy.load("es_core_news_sm")  #cargar el modelo en español

In [40]:
nltk.download('punkt')
nltk.download('stopwords')
stanza.download('es')
nlp_stanza = stanza.Pipeline('es')
nlp_spacy = spacy.load('es_core_news_sm')
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

[nltk_data] Downloading package punkt to C:\Users\John
[nltk_data]     Fredy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\John
[nltk_data]     Fredy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2025-08-20 21:20:31 INFO: Downloaded file to C:\Users\John Fredy\stanza_resources\resources.json
2025-08-20 21:20:31 INFO: Downloading default packages for language: es (Spanish) ...
2025-08-20 21:20:35 INFO: File exists: C:\Users\John Fredy\stanza_resources\es\default.zip
2025-08-20 21:20:52 INFO: Finished downloading models and saved to C:\Users\John Fredy\stanza_resources
2025-08-20 21:20:52 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2025-08-20 21:20:52 INFO: Downloaded file to C:\Users\John Fredy\stanza_resources\reso

In [31]:
stop_words = set(stopwords.words('spanish'))

In [44]:
pdf_path = "../Archivos/NTIC_SXXI.pdf"

## Funciones de limpieza de texto

In [45]:
def remove_header_by_position(pdf_path, header_height=100):
    """
    Elimina texto que aparece en la parte superior de cada página
    basándose en coordenadas Y (posición vertical)
    """
    extracted_text = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_height = page.height
            # Filtrar solo texto que NO esté en el área del encabezado
            filtered_text = ""
            
            if page.chars:  # Si hay caracteres en la página
                for char in page.chars:
                    # Si el carácter está por debajo del área del encabezado
                    if char['y1'] < (page_height - header_height):
                        filtered_text += char['text']
                        
            extracted_text.append(filtered_text)
    
    return " ".join(extracted_text)

In [46]:
def remove_footer_by_regex(text):
    """
    Elimina pie de página usando expresiones regulares específicas
    Parámetros:
        text (str): Texto del cual eliminar el pie de página
    Retorna:
        str: Texto sin pie de página
    """
    if not text:
        return ""
    
    # Patrón para el pie de página específico que mencionaste
    footer_pattern = r"http://www\.ugr\.es/~sevimeco/revistaeticanet/index\.htm\s*Rocío Domínguez Alfonso"
    
    # Patrones genéricos comunes en pies de página
    generic_footer_patterns = [
        r"Página\s+\d+\s+de\s+\d+",  # "Página X de Y"
        r"\d+\s*/\s*\d+",             # "1/10"
        r"www\.[^\s]+\.com",          # URLs web
        r"http[s]?://[^\s]+",         # URLs completas
        r"©\s*\d{4}.*",               # Copyright
        r"Todos los derechos reservados.*",
        r"Confidencial.*",
        r"\d{2}/\d{2}/\d{4}.*",       # Fechas
    ]
    
    # Eliminar pie de página específico
    cleaned_text = re.sub(footer_pattern, "", text, flags=re.IGNORECASE | re.DOTALL)
    
    # Eliminar patrones genéricos de pie de página
    for pattern in generic_footer_patterns:
        cleaned_text = re.sub(pattern, "", cleaned_text, flags=re.IGNORECASE | re.DOTALL)
    
    # Limpiar líneas vacías extras y espacios
    cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [ ]:
def preprocess_text(text):
    text = unicodedata.normalize("NFKD", text)  # Normalizar caracteres Unicode (ej. á → a)

    text = nlp(text.lower())  # Convertir todo el texto a minúsculas y procesarlo con spaCy

    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return tokens

In [48]:
raw_text = remove_header_by_position(pdf_path,150)  # Extraer texto limpio del PDF
pdf_text = remove_footer_by_regex(raw_text)  # Eliminar pies de página
tokens = preprocess_text(pdf_text)

print(tokens[:50])  # Mostrar los primeros 50 tokens como prueba

['Nuevas', 'Tecnologías', 'Educación', 'siglo', 'XXI', 'Rocío', 'Domínguez', 'Alfonso', 'Licenciada', 'Pedagogía', 'Doctoranda', 'Departamento', 'Didáctica', 'Organización', 'Escolar', 'Universidad', 'Granada', 'RESUMEN:', 'En', 'dos', 'últimas', 'décadas', 'autores', 'investigadores', 'acuñado', 'término', '“Sociedad', 'información”', 'referirse', 'serie', 'conjunto', 'transformaciones', 'económicas,', 'sociales,', 'culturales...', 'cambiarán', 'forma', 'sustancial', 'sociedad.', 'Quizá', 'transformación', 'más', 'espectacular', 'ofrecida', 'introducción', 'generalizada', 'nuevas', 'tecnologías', 'información', 'comunicación']


In [53]:
with open("../Resultado/taller_clase2.txt", "w", encoding="utf-8") as f:
    f.write(" ".join(pdf_text))

#Tokenización

In [28]:
# Download the necessary data for Spanish tokenization
nltk.download('punkt', quiet=True)
# nltk.download('spanish_grammars')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to C:\Users\John
[nltk_data]     Fredy\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [13]:
# 4. Tokenización con diferentes librerías

def tokenizar_spacy(texto):
    return [token.text for token in nlp_spacy(texto)]

def tokenizar_nltk(texto):
    return nltk.word_tokenize(texto)

def tokenizar_stanza(texto):
    doc = nlp_stanza(texto)
    return [word.text for sent in doc.sentences for word in sent.words]

def tokenizar_bert(texto):
    return tokenizer_bert.tokenize(texto)

In [ ]:
# Aplicamos cada método
tokens_spacy = tokenizar_spacy(pdf_text)
tokens_nltk = tokenizar_nltk(pdf_text)
tokens_stanza = tokenizar_stanza(pdf_text)
#tokens_bert = tokenizar_bert(texto_filtrado)

In [32]:
# Mostramos los primeros 20 tokens de cada método
print("\nTokens con SpaCy:", tokens_spacy[:20])
print("\nTokens con NLTK:", tokens_nltk[:20])
print("\nTokens con Stanza:", tokens_stanza[:20])


Tokens con SpaCy: ['Nuevas', 'Tecnologías', 'y', 'Educación', 'en', 'el', 'siglo', 'XXI', '  ', 'Rocío', 'Domínguez', 'Alfonso', 'Licenciada', 'en', 'Pedagogía', 'y', 'Doctoranda', 'del', 'Departamento', 'de']

Tokens con NLTK: ['Nuevas', 'Tecnologías', 'y', 'Educación', 'en', 'el', 'siglo', 'XXI', 'Rocío', 'Domínguez', 'Alfonso', 'Licenciada', 'en', 'Pedagogía', 'y', 'Doctoranda', 'del', 'Departamento', 'de', 'Didáctica']

Tokens con Stanza: ['Nuevas', 'Tecnologías', 'y', 'Educación', 'en', 'el', 'siglo', 'XXI', 'Rocío', 'Domínguez', 'Alfonso', 'Licenciada', 'en', 'Pedagogía', 'y', 'Doctoranda', 'de', 'el', 'Departamento', 'de']


In [23]:
def lemmatize_with_flair(text):
    """Lematizar texto usando Flair"""
    tagger = SequenceTagger.load('pos-fast')
    sentence = Sentence(text)
    tagger.predict(sentence)
    lemmas = [token.text for token in sentence.tokens]
    return lemmas

In [34]:
lemmas = lemmatize_with_flair(tokens)

2025-08-20 21:05:18,112 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [35]:
lemma_counts = Counter(lemmas).most_common(100)
df = pd.DataFrame(lemma_counts, columns=['Lema', 'Frecuencia'])

df.head(50)  

,Lema,Frecuencia
0,medios,25
1,más,22
2,Nuevas,21
3,formación,21
4,tecnologías,19
5,comunicación,18
6,Tecnologías,16
7,En,14
8,enseñanza,14
9,cambios,13


In [36]:
# Mostrar y guardar resultados
df.to_csv('../Resultado/lemmas_flair_top100.csv', index=False, encoding='utf-8-sig')

## NER

In [ ]:
# Introduce tu token aquí
token = ""

login(token=token)

tagger = SequenceTagger.load('flair/ner-spanish-large')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
e:\Universidad\Maestria\Semestre 4\Minería de texto\Codigo\clase-NLP\my-env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\John Fredy\.flair\models\ner-spanish-large\models--flair--ner-spanish-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrat

2025-08-20 21:30:02,098 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [42]:
# 3. Aplicar NER con Flair
def apply_ner(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = [(entity.text, entity.labels[0].value) for entity in sentence.get_spans('ner')]
    return entities


In [43]:
def count_entities(entities):
    counter = Counter(entities)
    data = [{'Entidad': ent, 'Tipo': typ, 'Frecuencia': freq} for (ent, typ), freq in counter.items()]
    df = pd.DataFrame(data)
    return df

In [49]:
entities = apply_ner(tokens)
result_df = count_entities(entities)

In [50]:
print(result_df)

                                          Entidad  Tipo  Frecuencia
0        Nuevas Tecnologías Educación siglo XXI  MISC           1
1                       Rocío Domínguez Alfonso   PER           1
2   Departamento Didáctica Organización Escolar   ORG           1
3                             Universidad Granada   ORG           1
4                                 Ortega Carrillo   PER           3
..                                            ...   ...         ...
85                             Escuela Española.   ORG           1
86                 Sociedad Española Pedagogía.   ORG           1
87                               Sevillano, Ma L.   PER           2
88             Tecnologías, medios comunicación  MISC           2
89                                           CCS.   ORG           2

[90 rows x 3 columns]


In [52]:
result_df.to_excel('../Resultado/NER_Frecuencias.xlsx', index=False)